In [1]:
import torch
import numpy as np
import pandas as pd
from sklearn import model_selection
import osi

In [11]:
import transformers

d:\NLP\bert-entity-extraction\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
## first file in current dir (with full path)
print(os.path.basename("input/multiconer1_2022/EN-English/en_test-2.conll").split(".")[0] + ".txt")
print(os.path.dirname(os.path.abspath("input/multiconer1_2022/EN-English/en_test-2.conll")))
print(os.path.abspath("input/multiconer1_2022/EN-English/en_test-2.conll"))
# file = os.path.join(os.getcwd(), os.listdir(os.getcwd())[0])
# file
# os.path.dirname(file) ## directory of file
# os.path.dirname(os.path.dirname(file)) ## directory of directory of file


en_test-2.txt
d:\NLP\bert-entity-extraction\input\multiconer1_2022\EN-English
d:\NLP\bert-entity-extraction\input\multiconer1_2022\EN-English\en_test-2.conll


In [10]:
x = [torch.tensor([1]), torch.tensor([2]), torch.tensor([3])]
print(x)
x = [tensor.item() for tensor in x]
print(x)
a = [torch.tensor([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])] * 3
print(a)
for i, z in enumerate(zip(a, x)):
    print(a[i][1:x[i]+1])


[tensor([1]), tensor([2]), tensor([3])]
[1, 2, 3]
[tensor([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10]), tensor([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10]), tensor([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10])]
tensor([2])
tensor([2, 3])
tensor([2, 3, 4])


In [12]:
TOKENIZER = transformers.BertTokenizer.from_pretrained(
    pretrained_model_name_or_path = "input/bert-base-uncased",
    do_lower_case=True
)

In [18]:
sentence = 'his playlist includes sonny sharrock, gza, country teasers and the notorious b. i. g.'
print(len(sentence.split()))
sentence = TOKENIZER.encode(sentence)
print(len(sentence))
print(sentence)
print(TOKENIZER.convert_ids_to_tokens(sentence))
sentence = TOKENIZER.decode(sentence)
print(sentence)

14
26
[101, 2010, 2377, 9863, 2950, 13584, 21146, 18933, 3600, 1010, 1043, 4143, 1010, 2406, 27071, 2015, 1998, 1996, 12536, 1038, 1012, 1045, 1012, 1043, 1012, 102]
['[CLS]', 'his', 'play', '##list', 'includes', 'sonny', 'sha', '##rro', '##ck', ',', 'g', '##za', ',', 'country', 'teaser', '##s', 'and', 'the', 'notorious', 'b', '.', 'i', '.', 'g', '.', '[SEP]']
[CLS] his playlist includes sonny sharrock, gza, country teasers and the notorious b. i. g. [SEP]


In [ ]:
import sklearn
from sklearn import metrics
import torch.nn as nn

from sklearn.metrics import classification_report
y_true = [0, 1, 2, 2, 2]
y_pred = [0, 0, 2, 2, 1]
target_names = ['class 0', 'class 1', 'class 2']
print(classification_report(y_true, y_pred, target_names=target_names))

y_true = [0, 1, 2, 2, 2]
y_pred = [0, 0, 2, 2, 1]
target_names = ['class 0', 'class 1', 'class 2']
print(classification_report(y_true, y_pred, target_names=target_names))


              precision    recall  f1-score   support

     class 0       0.50      1.00      0.67         1
     class 1       0.00      0.00      0.00         1
     class 2       1.00      0.67      0.80         3

    accuracy                           0.60         5
   macro avg       0.50      0.56      0.49         5
weighted avg       0.70      0.60      0.61         5



In [ ]:
df_csv = pd.read_csv("input/ner_dataset.csv", encoding="latin-1", keep_default_na=False, na_values=[''])
df_csv.loc[:, "Sentence #"] = df_csv["Sentence #"].fillna(method="ffill")
sentences = df_csv.groupby("Sentence #")["Word"].apply(list).values
print("Total sentence: " + df_csv["Sentence #"].iloc[-1])
(
    train_sentences,
    test_sentences,
) = model_selection.train_test_split(sentences, random_state=42, test_size=0.1)

print("Train len: "+ str(len(train_sentences)))
print("Val len: "+ str(len(test_sentences)))


C:\Users\PC\AppData\Local\Temp\ipykernel_20016\3054647275.py:2: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_csv.loc[:, "Sentence #"] = df_csv["Sentence #"].fillna(method="ffill")


Total sentence: Sentence: 47959
Train len: 43163
Val len: 4796


In [ ]:
def load_sentences_into_df(path): #load into a list of list to create df
    """
    Load sentences. A line must contain at least a word and its tag.
    Sentences are separated by empty lines.
    """
    sentences = []
    #sentence = [] #sentence: a list of line.split() for an actual sentence sample
    sentence_cnt = 0
    with open(path, encoding="utf8") as file:
        for line in file:
            #print(line)
            line = line.strip()
            if not line: #empty line to separate two sentences
                sentence_cnt += 1
            else: 
                line = line.split()
                sentence_word = [sentence_cnt]
                if len(line) != 4 or line[1] != "_":
                    #print(line)
                    continue
                else:
                    sentence_word.extend([line[0], line[-1]])
                    sentences.append(sentence_word)
    return pd.DataFrame(sentences, columns = ["Sentence #", "Word", "Tag"])
            

In [ ]:
sentences_dev = load_sentences_into_df("input/multiconer1_2022/EN-English/en_dev.conll")
train_df = load_sentences_into_df("input/multiconer1_2022/EN-English/en_train.conll")
test_df = load_sentences_into_df("input/multiconer1_2022/EN-English/en_test.conll")

In [ ]:
print(" ".join(filter(lambda x: "B-" in x, sentences_dev['Tag'].unique())))
sentences = sentences_dev.groupby("Sentence #")["Word"].apply(list).values
print(max(sentences, key=len))
longest = max(sentences, key=len)
print(len(longest))

print("train set")
sentences = train_df.groupby("Sentence #")["Word"].apply(list).values
longest = max(sentences, key=len)
print(longest)
print(len(longest))

print("test set")
sentences = test_df.groupby("Sentence #")["Word"].apply(list).values
longest = max(sentences, key=len)
print(longest)
print(len(longest))


B-CW B-PER B-CORP B-GRP B-LOC B-PROD
['the', 'bbc', 'did', 'a', 'news', 'story', ',', 'which', 'talked', 'about', 'how', 'the', 'causes', 'of', 'the', 'crisis', 'in', 'the', 'u.s.', 'economy', 'has', 'forced', 'many', 'people', ',', 'who', 'used', 'to', 'own', 'their', 'own', 'homes', ',', 'to', 'now', 'live', 'in', 'tents', '.']
39
train set
['in', '1981', ',', 'as', 'he', 'was', 'standing', 'in', 'mitel', '’s', 'lobby', 'with', 'one', 'of', 'the', 'company', '’s', '3', 'inch', 'wafers', 'in', 'his', 'hands', ',', 'light', 'hit', 'the', 'wafer', 'a', 'certain', 'way', 'and', 'he', 'saw', 'a', 'riot', 'of', 'colour', 'coming', 'off', '.']
41
test set
['aside', 'from', 'the', 'sd', '.', 'kfz', '.', '4', '/', '1', ',', 'the', 'sd', '.', 'kfz', '.', '4', 'was', 'armed', 'only', 'with', 'a', 'light', '7.92', 'mm', 'mg', '34', 'or', 'mg', '42', 'machine', 'gun', 'with', 'a', 'traverse', 'of', '270', '°', 'and', 'elevation', 'limits', 'of', '12', '°', 'to', '+', '80', '°', '.']
49


In [ ]:
def describe_df(df):
    print("-- Total sentences: " + str(df['Sentence #'].iloc[-1] + 1))
    #print("-- Set of Tags: " + " ".join(filter(lambda x: "B-" in x, df['Tag'].unique())))
    print("-- Set of Tags: " + " ".join(df['Tag'].unique()) + " -> " +str(df['Tag'].nunique()))
    print("-- PER (person): " + str(df['Tag'].loc[df['Tag'].str.contains("B-PER")].count()))
    print("-- LOC (location): " + str(df['Tag'].loc[df['Tag'].str.contains("B-LOC")].count()))
    print("-- CORP (corporation): " + str(df['Tag'].loc[df['Tag'].str.contains("B-CORP")].count()))
    print("-- GRP (group): " + str(df['Tag'].loc[df['Tag'].str.contains("B-GRP")].count()))
    print("-- PROD (product): " + str(df['Tag'].loc[df['Tag'].str.contains("B-PROD")].count()))
    print("-- CW (creative work): " + str(df['Tag'].loc[df['Tag'].str.contains("B-CW")].count()))

In [ ]:
print("Training set")
describe_df(df_train)
print("Validation set")
describe_df(df_dev)
print("Test set")
describe_df(df_test)

Training set
-- Total sentences: 15300
-- Set of Tags: O B-PER I-PER B-GRP I-GRP B-CW I-CW B-LOC B-CORP I-CORP B-PROD I-LOC I-PROD -> 13
-- PER (person): 5397
-- LOC (location): 4799
-- CORP (corporation): 3111
-- GRP (group): 3571
-- PROD (product): 2923
-- CW (creative work): 3752
Validation set
-- Total sentences: 800
-- Set of Tags: B-CW I-CW O B-PER I-PER B-CORP I-CORP B-GRP I-GRP B-LOC I-LOC B-PROD I-PROD -> 13
-- PER (person): 290
-- LOC (location): 234
-- CORP (corporation): 193
-- GRP (group): 190
-- PROD (product): 147
-- CW (creative work): 176
Test set
-- Total sentences: 217818
-- Set of Tags: B-PROD I-PROD O B-GRP B-LOC I-LOC I-GRP B-CORP I-CORP B-CW B-PER I-PER I-CW -> 13
-- PER (person): 55682
-- LOC (location): 59082
-- CORP (corporation): 37435
-- GRP (group): 41156
-- PROD (product): 36786
-- CW (creative work): 42781
